<a href="https://colab.research.google.com/github/khaoulaloudiyi/QuickNews/blob/master/start_media_insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/MediaInsight

/content/drive/.shortcut-targets-by-id/1slwd6zf9z3HwT_o9xa6gz2ym2VnOfDD8/MediaInsights


In [ ]:
!pip install BeautifulSoup4
!pip install pandas
!pip install datetime
!pip install dateparser
!pip install flask_ngrok
!pip install googletrans


     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 235kB 7.3MB/s 
     |████████████████████████████████| 368kB 2.7MB/s 
     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 962kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=a481bdcffddf1f165b387e06b4dabd0523cc5b1b060089698ea7d295010267a5
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=1d980411221c97f41bc92abade6278bbda0cab7c97a578dc6063f1490b10e2ec
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f

In [ ]:
from collections import Counter 
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
swa = stopwords.words('arabic')
swf = set(stopwords.words('french'))
sws = set(stopwords.words('spanish'))
#tokenizing sentences into words
def tokenizer(s):
    tokens = []
    for word in s.split(' '):
        tokens.append(word.strip().lower()) #strip remove spaces at the beginning and at the end of the string
    return tokens
#tokenizing document into sentences 

def sent_tokenizer(s):
    sents = []
    for sent in s.split('.'):
        sents.append(sent.strip())
    return sents


def get_content(url):
  req_obj = requests.get(url)
  text = req_obj.text
  soup = BeautifulSoup(text)
  title = soup.find("div" , {"class" : "col article-container"})
  content_a = soup.findAll("div", {"class": "content"})
  content = ''
  for x in content_a:
    for c in x.findAll('p'):
      content += c.text + "\n"
  return content


def count_words(tokens):
    word_counts = {}
    for token in tokens:
        if token not in stop_words and token not in STOP_WORDS and token not in swa and token not in sws and token not in swf and token not in punctuation:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts



def word_freq_distribution(word_counts):
    freq_dist = {}
    max_freq = max(word_counts.values())
    for word in word_counts.keys():  
        freq_dist[word] = (word_counts[word]/max_freq)
    return freq_dist

def score_sentences(sents, freq_dist, max_len=40):
    sent_scores = {}  
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() in freq_dist.keys():
                if len(words) < max_len:
                    if sent not in sent_scores.keys():
                        sent_scores[sent] = freq_dist[word.lower()]
                    else:
                        sent_scores[sent] += freq_dist[word.lower()]
    return sent_scores


def summarize(sent_scores, k):
    top_sents = Counter(sent_scores) 
    summary = ''
    scores = []
    
    top = top_sents.most_common(k)
    for t in top: 
        summary += t[0].strip()+'. '
        scores.append((t[1], t[0]))
    return summary[:-1], scores


def function(url_content):
  tokens = tokenizer(url_content)
  sents = sent_tokenizer(url_content)
  word_counts = count_words(tokens)
  freq_dist = word_freq_distribution(word_counts)
  sent_scores = score_sentences(sents, freq_dist)
  summary, summary_sent_scores = summarize(sent_scores, 3)
  return summary



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from googletrans import Translator
translator = Translator()


In [ ]:
 #pip install BeautifulSoup4, pandas, datetime, dateparser

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from flask import Flask, request, render_template, g 
from bs4 import BeautifulSoup
import urllib.request
from flask_ngrok import run_with_ngrok
from flask_ngrok import *
from urllib.request import Request, urlopen
from datetime import datetime
import pandas as pd
import csv
import dateparser
import sqlite3
import json 
import re

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run
article=""

#recherche 1
def score_content(my_theme, content):
    result = []
    vectorizer = TfidfVectorizer(stop_words = "english", use_idf = True, smooth_idf = True)
    vectorizer.fit(content+my_theme)
    vector_list_contents = vectorizer.transform(content)
    vector_my_theme = vectorizer.transform(my_theme)
    result = linear_kernel(vector_my_theme, vector_list_contents)
    score=sum(result)/len(result)
    return score

def Recommendation(my_theme, list_content, dateFrom, dateTo):
    themes_publishers = []
    for k in range(0, len(list_content['Id'])):
        S= [1]
        if dateFrom or dateTo:
            if dateFrom and dateparser.parse(dateFrom)> dateparser.parse(list_content['Date'][k]):
                S= [0]
            if dateTo and dateparser.parse(dateTo)< dateparser.parse(list_content['Date'][k]):
                S= [0]
        if S[0] == 1:
            S = score_content([my_theme], [list_content['Content'][k]])
        themes_publishers.append(S[0])
    list_content['score']=themes_publishers 
    list_content.sort_values(by=["score"] , inplace=True , ascending = False )
    
    articles_result = []
    artic = pd.DataFrame(columns=['Id','Title','Author','Date', 'Publication','URL','Content','Language','Country','sentiment','Summary'])
    if(list_content.loc[list_content.index[0],"score"]< 0.001):
        return articles_result, False
    else:
        #print("\nLes TOP articles traitants votre theme sont :\n")
        e = 0
        for i, element in list_content.iterrows():
            if(element["score"] >= 0.001) or e==50:
                my_dict = {'publication' : element['Publication'], 'titre' : element['Title'], 'date' : element['Date'], 'content' : element['Content'][0:140]+"...", 'author' : element['Author'], 'URL': element['URL'], 'sentiment' : element['sentiment'], 'summary': element['Summary']}
                articles_result.append(my_dict)
                artic = artic.append(element, ignore_index=True)
                e = e+1
            else:
                break    
    return articles_result, artic

def research_by_similarity(my_theme, dateFrom, dateTo):
    global article
    list_content=pd.read_csv(article)
    list_content = list_content #[:100]
    return Recommendation(my_theme, list_content, dateFrom, dateTo)

#recherche2
def syntaxique_research(list_inputs,dateFrom, dateTo):
    data = pd.read_csv(article, low_memory=False, encoding='utf-8')
    answer=False
    articles = []
    if dateFrom :
            dateF = dateparser.parse(dateFrom)
    if dateTo:
            dateT = dateparser.parse(dateTo) 
    artic = pd.DataFrame(columns=['Id','Title','Author','Date', 'Publication','URL','Content','Language','Country','sentiment','Summary']) 
    for i in range(len(data.iloc[:,2])):
        if dateFrom :
            if dateF > dateparser.parse(data.loc[i,'Date']):
                continue
        if dateTo:
            if  dateT< dateparser.parse(data.loc[i,'Date']):
                continue
        for j in list_inputs:
            e=j.lower()
            if type(data.loc[i,'Content']) is str:
                res = data.loc[i,'Content'].lower().find(e)
            else :
                res = -1

            if res != -1 :
                my_dict = {'publication' : data.loc[i,'Publication'], 'titre' : data.loc[i,'Title'], 'date' : data.loc[i,'Date'], 'content' : data.loc[i,'Content'][0:140]+"...", 'author' : data.loc[i,'Author'], 'URL': data.loc[i,'URL'], 'sentiment' : data.loc[i,'sentiment'], 'summary': data.loc[i,'Summary'] }
                articles.append(my_dict)
                artic = artic.append(data.loc[i], ignore_index=True)
                answer= True
                break
    return articles, artic

def pre_process(text):
    
    text=text.lower()
    text=re.sub("</?.*?>"," <> ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    return text

def get_stop_words():
    #with open('stop_words.csv', 'r', encoding="utf-8") as f:
    data = pd.read_csv('stopWords.csv', low_memory=False, encoding='utf-8')
    stopwords = data.loc[:,"stop_words"]
    stop_set = set(m.strip() for m in stopwords)
    return frozenset(stop_set)


def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

def get_Tendance(articles):
    my_list = articles['Content'].tolist()
    final_element = ''
    #gather all the elements in one element
    for i in range(len(my_list)):
        final_element = final_element + " "+ pre_process(my_list[i])
    # get all the stop_words from spanish, english, frensh and arabic
    stopwords=get_stop_words()
    #generate tf-idf for the given document
    vectorizer = TfidfVectorizer(stop_words = stopwords, use_idf=True, smooth_idf=True)
    cv = vectorizer.fit(my_list)
    feature_names = cv.get_feature_names()
    tf_idf_vector = vectorizer.transform([final_element])


    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract the top n keywords
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    #results
    keyWordList = []
    for k in keywords:
        myDict = {"Name":k, "Count":int(keywords[k]*10000)}
        #word = []
        #word.append(str(k))
        #word.append(int(keywords[k]*1000))
        keyWordList.append(myDict)
    keyWords ={"children": keyWordList}
    return  keyWords

@app.route('/research', methods=['POST'])
def get_articles():
    global article
    dateFrom = request.form['from']
    dateTo = request.form['to']
    list_inputs = [" "+a.strip().lower()+" " for a in request.form['keyword'].split("-")]
    if request.form['gridRadios1']  == 'option21':
      article="ArticlesEP.csv"
    elif request.form['gridRadios1']  == 'option31':
      article="ArticlesAR.csv"
    elif request.form['gridRadios1']  == 'option41':
      article="Articles.csv"
    else:
      article="ArticlesENG.csv"
    if request.form['gridRadios']  == 'option1':
      articles, artic = syntaxique_research(list_inputs, dateFrom, dateTo)
    else :
      articles, artic = research_by_similarity(" ".join(list_inputs), dateFrom, dateTo)
    if not artic.empty:
        maxD = {}
        values, keys =list(artic.groupby('Country')['Country'].transform('count')), artic['Country'].tolist()
        countrydictionary = dict(zip(keys, values))
        maxD["max"] =  sum(countrydictionary.values())
        
        authordictionary= {}
        values, keys =list(artic.groupby('Author')['Author'].transform('count')), artic['Author'].tolist()
        authordictionary = dict(zip(keys, values))
        sorted_author = sorted(authordictionary.items(), key=lambda kv: kv[1], reverse=True)
        sumAuthor = sum(authordictionary.values())
        authorList = []
        count = 0
        for i in range(0,min(9,len(sorted_author))):
            authorList.append(sorted_author[i])
            count += sorted_author[i][1]
        authorList.append(["others", sumAuthor - count])
        authordictionary["authors"] = authorList

        values, keys =list(artic.groupby('sentiment')['sentiment'].transform('count')), artic['sentiment'].tolist()
        sentimentdictionary = dict(zip(keys, values)) 
    
        #values, keys =list(artic.groupby('summary')['summary'].transform('count')), artic['summary'].tolist()
        #summarydictionary = dict(zip(keys, values))  

        tendance= get_Tendance(artic)
        
        message = "Le nombre de résultat est : "+str(len(articles))
        showRes = "true"
        
        return render_template('Show_stat.html', countrydict = json.dumps(countrydictionary), authordict=json.dumps(authordictionary), sentimentdict=json.dumps(sentimentdictionary), maxD = json.dumps(maxD), articles=articles, num = message, keyWords = json.dumps(tendance), showRes = showRes)
    else:
        message = "Out of InSight "
        showRes = "false"
        return render_template('Show_stat.html', countrydict = json.dumps({}), authordict=json.dumps({}), sentimentdict=json.dumps({}), maxD = json.dumps(0), articles=[], num = message, keyWords = json.dumps({}), showRes = showRes)


    #return render_template('search_presse.html', articles=articles, num = "Le nombre de résultat est : "+str(len(articles)))

@app.route('/') 
def get_root():
    return render_template('index1.html')

@app.route('/SignUp.html')
def get_sign2():
    return render_template('SignUp.html')

@app.route('/Login.html')
def get_login():
    return render_template('Login.html')

@app.route('/Show_stat.html')
def get_stat():
    return render_template('Show_stat.html')

@app.route('/accueil')
def get_ac():
    return render_template('index1.html')


@app.route('/search_presse.html')
def get_research_():
    return render_template('search_presse.html')


@app.route('/index.html',methods=['GET', 'POST'])
def get_resume():
  if request.method == "POST": 
    url = request.form.get("url")
    summary = request.form.get("summary")

    summary_ar = (translator.translate(summary,dest='ar')).text
    summary_en = (translator.translate(summary,dest='en')).text
    summary_sp = (translator.translate(summary,dest='es')).text
    summary_fr = (translator.translate(summary,dest='fr')).text

    publication = request.form.get("publication")
    titre = request.form.get("titre")

    
  return render_template('index.html',summary=summary,publication=publication,titre=titre)
  

 
    
@app.route('/existing_account', methods=['POST'])
def existing_account():
    g.db = sqlite3.connect('library.db')
    users = g.db.execute("SELECT name FROM Users where name ='"+request.form['name']+"' and password = '"+request.form['pass']+"';")
    exist = False
    for user in users.fetchall():
        exist = True
    if not exist:
        return render_template('Login.html', message = "Compte n'existe pas")
    else:
        return render_template('search_presse.html')

@app.route('/new_account', methods=['POST'])
def new_account():
    g.db = sqlite3.connect('library.db')
    users = g.db.execute("SELECT name FROM Users where name ='"+request.form['name']+"' and password = '"+request.form['pass']+"';")
    exist = False
    for user in users.fetchall():
        exist = True
    if not exist:
        if request.form['pass'] == request.form['re_pass'] :
            g.db.execute("Insert Into Users(name, password, email) Values('"+ request.form['name'] +"','"+ request.form['pass'] +"','"+request.form['email'] +"');")
            g.db.commit()
            return render_template('search_presse.html')
        else :
            return render_template('SignUp.html', message = "Le mot de passe doit être <br> identique !!!")
    else:
        return render_template('SignUp.html', message = "User name existe déja")

if __name__ == "__main__":
  print("Start running server")
  app.run()


Start running server
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4333ca9d88b5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Sep/2020 13:49:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:00] "GET /static/img/core-img/logo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:00] "GET /static/img/bg-img/18.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:00] "GET /static/img/bg-img/17.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:00] "GET /static/img/bg-img/16.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:09] "GET /search_presse.html HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:18] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:50:20] "GET /static/img/core-img/logo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:52:40] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:52:41] "GET /static/css/animate.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:52:41] "GET /static/css/default-assets/classy-nav.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 13:52:42] "GET /static/js/popper.min.js HTTP/1.1" 200 -
127.0

In [ ]:
pip install --upgrade git+https://github.com/dbcli/mycli.git